# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Import packages
import nltk
nltk.download('words')
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

import re
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine

[nltk_data] Downloading package words to /home/beililiu/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/beililiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/beililiu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/beililiu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/beililiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/beililiu/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable', engine)
X = df.message
Y = df.iloc[:, 4:]

In [4]:
i = 13
print(X[i])
print(Y.iloc[i, :])

Let's do it together, need food in Delma 75, in didine area
related                   1
request                   1
offer                     0
aid_related               1
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      1
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold            

In [5]:
def tokenize(text):
    '''
        Convert a string into a list of words. English stop words
        are removed and all words are reduced to their base form
        using lemmatization.
        
        Arguments:
        text - a string
        
        Returns:
        tokens - a list of word tokens derived from the text    
    '''
    
    import re
    # Remove punctuation and special characters
    text = re.sub('[^\w\s]', '', text)
    # Make text lower case
    text = text.lower().strip()
    # Tokenize the sentence
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if word not in nltk.corpus.stopwords.words("english")]
    # Lemmatize each word
    tokens = [WordNetLemmatizer().lemmatize(word, pos='v') for word in tokens]
    
    return tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split the model into a training and test set

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
# Train an RF model using a multioutput classifier

from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
                     ('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=50)))
                    ])

pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
# Create a function to score the model

def score_model(X_test, Y_test, model):
    '''
        Score a model using test data by calculating the f1 score,
        precision, and recall. The scores are printed to the console.
        
        Arguments:
        X_test - the test data
        Y_test - the test target
        model - the model to be scored
        
        Returns:
        None
    '''
    
    from sklearn.metrics import classification_report
    
    # Predict labels for each message category
    Y_pred = model.predict(X_test)
    labels = Y.columns.values.tolist()
    Y_pred_df = pd.DataFrame(Y_pred, columns=labels)
    
    # Calculate the scoring metrics and print them to console
    for col_pred, col_true in zip(Y_pred_df, Y_test):
        print(col_pred)
        print(classification_report(Y_test[col_true], Y_pred_df[col_pred]))

score_model(X_test, Y_test, pipeline)

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
# Search for parameters to improve the model performance
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
                     ('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=50)))
                    ])

parameters = {
              'vect__ngram_range': ((1, 1), (1, 2)),
              'vect__max_df': (0.5, 0.75, 1.0),
              'tfidf__smooth_idf': (True, False),
              'tfidf__sublinear_tf': (True, False),
              'clf__min_samples_leaf': (1, 2, 4),
              'clf__max_features': ('auto', 'log2')
             }

cv = GridSearchCV(pipeline, scoring='f1_samples', param_grid=parameters, cv=3, n_jobs=-1)
cv.fit(X_train, Y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 0.75, 1.0), 'tfidf__smooth_idf': (True, False), 'tfidf__sublinear_tf': (True, False), 'clf__min_samples_leaf': (1, 2, 4), 'clf__max_features': ('auto', 'log2')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_samples', verbose=0)

In [11]:
cv.best_params_

{'clf__max_features': 'auto',
 'clf__min_samples_leaf': 2,
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': True,
 'vect__max_df': 0.75,
 'vect__ngram_range': (1, 2)}

In [10]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(cv, open(filename, 'wb'))


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
score_model(X_test, Y_test, cv)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [44]:
# Test out different models to see if there is a performance improvement
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

pipeline_models = Pipeline([
                     ('vect', CountVectorizer(max_df=0.75, ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=True)),
                     ('clf', MultiOutputClassifier(estimator=None))
                    ])

parameters = [
                {
                    'clf__estimator': [MultinomialNB()],
                    'clf__estimator__alpha': (0.1, 0.5, 1.0)
                }
                {
                    'clf__estimator': [AdaBoostClassifier()],
                    'clf__estimator__learning_rate': (0.1, 0.5, 1.0)
                }
            ]

cv_models = GridSearchCV(pipeline_models, scoring='f1_samples', param_grid=parameters, cv=3, n_jobs=-1)
cv_models.fit(X_train.values, Y_train.values)

/Users/beili/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/Users/beili/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/beili/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/Users/beili/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/Users/beili/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-sc

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.75, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        stri...rue, sublinear_tf=True, use_idf=True)), ('clf', MultiOutputClassifier(estimator=None, n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'clf__estimator': [MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)], 'clf__estimator__alpha': (0.1, 0.5, 1.0)}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_samples', verbose=0)

In [30]:
score_model(X_test, Y_test, cv_models)

related
              precision    recall  f1-score   support

           0       0.68      0.12      0.21      1242
           1       0.78      0.98      0.87      3994

   micro avg       0.78      0.78      0.78      5236
   macro avg       0.73      0.55      0.54      5236
weighted avg       0.76      0.78      0.71      5236

request
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      4374
           1       0.78      0.47      0.59       862

   micro avg       0.89      0.89      0.89      5236
   macro avg       0.84      0.72      0.76      5236
weighted avg       0.88      0.89      0.88      5236

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5206
           1       0.00      0.00      0.00        30

   micro avg       0.99      0.99      0.99      5236
   macro avg       0.50      0.50      0.50      5236
weighted avg       0.99      0.99      0.99      523

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      4236
           1       0.74      0.34      0.46      1000

   micro avg       0.85      0.85      0.85      5236
   macro avg       0.80      0.66      0.69      5236
weighted avg       0.84      0.85      0.83      5236



/Users/beili/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
## Add features
from sklearn.base import BaseEstimator, TransformerMixin

# This estimator creates two features. One feature is an
# indicator for the word water and the other is an
# indicator for the word food
class WaterFoodExtractor(BaseEstimator, TransformerMixin):

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_water = X.str.contains('water')
        X_water.name = 'water'
        X_food = X.str.contains('food')
        X_food.name = 'food'
        
        return pd.concat([X_water, X_food], axis=1)

from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import FeatureUnion

pipeline_models = Pipeline([
                            ('features', FeatureUnion([
                                ('nlp_pipeline', Pipeline([
                                    ('vect', CountVectorizer(tokenizer=tokenize, max_df=0.75, ngram_range=(1, 2))),
                                    ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=True)),
                                ])),
                                ('water_food_extractor', WaterFoodExtractor())           
                            ])),
                            ('clf', MultiOutputClassifier(AdaBoostClassifier(learning_rate=0.5)))
                           ])

pipeline_models.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max...or=None,
          learning_rate=0.5, n_estimators=50, random_state=None),
           n_jobs=None))])

In [27]:
score_model(X_test, Y_test, pipeline_models)

related
              precision    recall  f1-score   support

           0       0.67      0.13      0.21      1242
           1       0.78      0.98      0.87      3994

   micro avg       0.78      0.78      0.78      5236
   macro avg       0.73      0.55      0.54      5236
weighted avg       0.76      0.78      0.72      5236

request
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      4374
           1       0.78      0.48      0.59       862

   micro avg       0.89      0.89      0.89      5236
   macro avg       0.84      0.72      0.76      5236
weighted avg       0.88      0.89      0.88      5236

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5206
           1       0.00      0.00      0.00        30

   micro avg       0.99      0.99      0.99      5236
   macro avg       0.50      0.50      0.50      5236
weighted avg       0.99      0.99      0.99      523

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5142
           1       0.70      0.22      0.34        94

   micro avg       0.98      0.98      0.98      5236
   macro avg       0.84      0.61      0.67      5236
weighted avg       0.98      0.98      0.98      5236

other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4937
           1       0.63      0.06      0.10       299

   micro avg       0.94      0.94      0.94      5236
   macro avg       0.79      0.53      0.54      5236
weighted avg       0.93      0.94      0.92      5236

direct_report
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      4236
           1       0.74      0.34      0.46      1000

   micro avg       0.85      0.85      0.85      5236
   macro avg       0.80      0.65      0.69      5236
weighted avg       0.84      0.85      0.83   

/Users/beili/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [31]:
import pickle
# save the model to disk
filename = 'final_model.sav'
pickle.dump(pipeline_models, open(filename, 'wb'))

# load the model from disk
# cv = pickle.load(open(filename, 'rb'))

In [29]:
cv.best_params_


{'clf__estimator__learning_rate': 0.5}

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.